In [17]:
# Open the Parlamint file
import pandas as pd

test_df = pd.read_csv("/home/tajak/Parlamint-translation/Testing-MT-models/ParlaMint-SI/Slovene-sample-for-manual-analysis-aligned-corrected-SouthSlavicMT.csv", sep="\t", index_col = 0)
test_df.head()

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments,new_translations,substitution_info,substituted_words,errors
Column1,,,,,,,,,,,,
443123,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg1.1,"Spoštovani , prosim , da zasedete svoja mesta .",0,9,"Dears, please take your seats.","Dears , please take your seats .","{0: 0, 1: 1, 2: 2, 4: 3, 6: 4, 7: 5, 8: 6}","Dears , please take your seats .",[],[],No
443124,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg2.1,V naši sredini pozdravljam predsednika države ...,"{8: ['Boruta', 'Borut'], 9: ['Pahorja', 'Pahor...",19,"In our midst, I welcome the President of the S...","In our midst , I welcome the President of the ...","{0: 0, 1: 1, 2: 2, 3: 5, 4: 7, 5: 10, 6: 11, 7...","In our midst , I welcome the President of the ...","[""No substitution: ('Borut', 'Borut')"", ""No su...",[],No
443125,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg2.2,Lepo pozdravljena .,0,3,"Well, hello there.","Well , hello there .","{0: 0, 1: 2, 2: 4}","Well , hello there .",[],[],No
443126,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg3.1,Besedo dajem spoštovanemu predsedniku Državneg...,"{8: ['Janku', 'Janko'], 9: ['Vebru', 'Veber']}",11,I give the Word to the Honorable President of ...,I give the Word to the Honorable President of ...,"{0: 3, 1: 4, 2: 6, 3: 7, 4: 10, 5: 11, 6: 12, ...",I give the Word to the Honorable President of ...,"[""No substitution: ('Janko', 'Janko')"", ('Vebr...","[('Vebr', 'Veber')]",No
443127,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg3.2,"Izvolite , prosim .",0,4,"Here you go, please.","Here you go , please .","{0: 2, 1: 3, 2: 4, 3: 5}","Here you go , please .",[],[],No


In [18]:
test_df = test_df.drop(columns=['translation',
       'translation_added_spaces', 'alignments', 'new_translations',
       'substitution_info', 'substituted_words', 'errors'])

test_df.head(2)

,file,sentence_id,text,proper_nouns,length
Column1,,,,,
443123,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg1.1,"Spoštovani , prosim , da zasedete svoja mesta .",0,9
443124,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg2.1,V naši sredini pozdravljam predsednika države ...,"{8: ['Boruta', 'Borut'], 9: ['Pahorja', 'Pahor...",19


Translate the text using the Slavic model which was shown to be the best for Slovene

In [19]:
def translate(opus_lang_code, df):
	"""
	This function translates the text from the dataframe, created with the extract_text() function
	with OPUS-MT models using EasyNMT. It returns a dataframe with the translation.

	Args:
	- opus_lang_code: the lang code to be used in the OPUS-MT model - use the one that performed the best in the comparison (see function choose_model())
	"""
	import pandas as pd
	import regex as re
	from easynmt import EasyNMT
	from IPython.display import display
	import time

	# Define the model
	model = EasyNMT('opus-mt')

	print("Entire corpus has {} sentences and {} words.".format(df["text"].count(), df["length"].sum()))

	# Create a list of sentences from the df
	sentence_list = df.text.to_list()

	print("Translation started.")

	start_time = time.time()

	#Translate the list of sentences - you need to provide the source language as it is in the name of the model - the opus_lang_code
	translation_list = model.translate(sentence_list, source_lang = "{}".format(opus_lang_code), target_lang='en')

	translation_time = round((time.time() - start_time)/60,2)

	print("Translation completed. It took {} minutes for {} instances - {} minutes per one sentence.".format(translation_time, len(sentence_list), translation_time/len(sentence_list)))

	# Add the translations to the df
	df["translation"] = translation_list

	# Display the df
	print(df[:3].to_markdown())

	return df

test_df = translate("sla", test_df)

Entire corpus has 41 sentences and 616 words.
Translation started.


/home/tajak/Parlamint-translation/parlamint_env/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translation completed. It took 0.4 minutes for 41 instances - 0.00975609756097561 minutes per one sentence.
|   Column1 | file                                          | sentence_id                                  | text                                                                                                                          | proper_nouns                                                                                      |   length | translation                                                                                                                   |
|----------:|:----------------------------------------------|:---------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------|---------:|:--------------------------------------------------------------------

Tokenize the English translation and substitute the incorrectly translated proper nouns.

In [20]:
translation_list = test_df.translation.to_list()

import re

translation_added_spaces = []

for translation in translation_list:
  # Add spaces around each punctuation
  translation = re.sub('([.,!?:;()])', r' \1 ', translation)
  # Remove duplicated spaces
  translation = re.sub('\s{2,}', ' ', translation)
  translation_added_spaces.append(translation)

test_df["translation_added_spaces"] = translation_added_spaces

test_df.head()

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces
Column1,,,,,,,
443123,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg1.1,"Spoštovani , prosim , da zasedete svoja mesta .",0,9,"Respectable, please take your seats.","Respectable , please take your seats ."
443124,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg2.1,V naši sredini pozdravljam predsednika države ...,"{8: ['Boruta', 'Borut'], 9: ['Pahorja', 'Pahor...",19,In our midst I welcome the President of the St...,In our midst I welcome the President of the St...
443125,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg2.2,Lepo pozdravljena .,0,3,Nicely greeted.,Nicely greeted .
443126,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg3.1,Besedo dajem spoštovanemu predsedniku Državneg...,"{8: ['Janku', 'Janko'], 9: ['Vebru', 'Veber']}",11,I give the floor to the distinguished Presiden...,I give the floor to the distinguished Presiden...
443127,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg3.2,"Izvolite , prosim .",0,4,"Here you go, please.","Here you go , please ."


In [21]:
def correct_proper_nouns(df):
	"""
	This function takes the translated text and the source text, aligns words with eflomal and corrects proper nouns.
	It takes the dataframe that was created in the function extract_text() and to which the translation was added
	in the function translate().

	To use eflomal, you need to install it first:
	!git clone https://github.com/robertostling/eflomal
	%cd eflomal
	!make
	!sudo make install
	!python3 setup.py install

	In case you don't have sudo permission, you can skip !sudo make install. I did, and I also used a virtual environment (venv), and managed to install eflomal.

	"""
	import pandas as pd
	import re
	import ast
	from IPython.display import display
	import time
	import subprocess
	import os

	# Move into the eflomal folder
	os.chdir("/home/tajak/Parlamint-translation/eflomal")

	# Then we need to create files for all texts and all translations
	source_sentences = open("source_sentences.txt", "w")
	English_sentences = open("English_sentences.txt", "w")

	for i in df["text"].to_list():
		source_sentences.write(i)
		source_sentences.write("\n")

	for i in df["translation_added_spaces"].to_list():
		English_sentences.write(i)
		English_sentences.write("\n")

	source_sentences.close()
	English_sentences.close()

	print("\n\n")
	print("Alignment started.")
	start_time = time.time()
	
	# Align sentences with eflomal and get out a file with alignments
	#!python3 align.py -s source_sentences.txt -t English_sentences.txt --model 3 -r source-en.rev -f source-en.fwd
	# Use the Slovene-English priors
	# Now, if you have another file to align, sl-en.small, simply use:
	!python3 align.py -s source_sentences.txt -t English_sentences.txt --priors sl-en.priors --model 3 -r source-en.rev

	# Create forward target alignments from the source alignment direction (by changing the direction in the rev file)
	aligns_list = open("source-en.rev", "r").readlines()
	aligns_list = [i.replace("\n", "") for i in aligns_list]

	# Continue with processing the list to create the final alignments format which I'll use to correct proper names
	aligns_list = [i.split(" ") for i in aligns_list]

	for i in aligns_list:
		for pair in i:
			current_pair = pair.split("-")
			i[i.index(pair)] = {int(current_pair[0]): int(current_pair[1])}
	
	final_aligns = []

	# Create a dictionary out of the rev alignments
	for i in aligns_list:
		current_line = {}

		try:
			for element in i:
				a = list(element.items())[0][0]
				b = list(element.items())[0][1]
				current_line[a] = b
		
			# Check whether the number of pairs in the list is the same as number of items
			if len(i) != len(list(current_line.items())):
				print("Not okay:")
				print(i)
				print(current_line)

			final_aligns.append(current_line)
		
		except:
			print("error")
			print(aligns_list.index(i))
			print(i)
			final_aligns.append("Error")
		
	print("\nNumber of aligned sentences: {}\n\n".format(len(final_aligns)))

	# Add a to the df
	df["alignments"] = final_aligns

	# Remove the rev and fwd file
	os.remove("source-en.rev")

	# When we open the dataframe file, the dictionaries with proper names changed into strings - Change strings in the column proper_nouns into dictionaries

	df["proper_nouns"] = df.proper_nouns.astype("str")
	df["proper_nouns"] = df.proper_nouns.apply(lambda x: ast.literal_eval(x))

	# Change nan values in the proper_nouns columns
	df = df.fillna(0)

	# Substitute words in the translation based on alignments
	intermediate_list = list(zip(df["translation_added_spaces"], df["proper_nouns"], df["alignments"]))

	new_translations = []
	substituted_all_info = []
	substituted_only = []
	substituted_words = []

	# Add information whether an error occurred
	error_list = []

	for i in intermediate_list:
		current_substituted_list = []
		current_substituted_only = []
		current_substituted_words = {}
		current_error = "No"

		# If no proper names were detected, do not change the translation
		if i[1] == 0:
			new_translations.append(i[0])
		
		else:
			current_translation = i[0]

			# Substitute the word with the source lemma based on the index - loop through the proper nouns to be changed
			for word_index in list(i[1].keys()):
				try:
					# split the translation into list of words
					word_list = current_translation.split()

					# Get index of the substituted word
					substituted_word_index = i[2][word_index]

					# Get the lemma to substitute the word with
					correct_lemma = i[1][word_index][1]

					# If the substitute word and lemma are not the same, get substituted word and its match
					if word_list[substituted_word_index] != correct_lemma:
						current_substituted_list.append((word_list[substituted_word_index], correct_lemma))
						current_substituted_only.append((word_list[substituted_word_index], correct_lemma))

						# Save information on which word was substituted with its conllu index (index + 1) as the key
						current_substituted_words[int(substituted_word_index+1)] = word_list[substituted_word_index]

						# Substitute the word in the word list
						word_list[substituted_word_index] = correct_lemma
					
					else:
						# Add information that substitution was not performed
						current_substituted_list.append(f"No substitution: {word_list[substituted_word_index], correct_lemma}")
					
					# Change the translation by merging the words back into a string
					current_translation = " ".join(word_list)

				except:
					print(f"Issue: index {word_index}: {i[1][word_index]}")
					current_error = f"Issue: index {word_index}: {i[1][word_index]}"

			# After the loop through proper nouns, save the new translation
			new_translations.append(current_translation)
		
		# Add information on what was substituted
		if len(substituted_all_info) != 0:
			substituted_all_info.append(current_substituted_list)
		else:
			substituted_all_info.append(0)

		if len(current_substituted_only) != 0:
			substituted_only.append(current_substituted_only)
		else:
			substituted_only.append(0)

		error_list.append(current_error)

		substituted_words.append(current_substituted_words)


	# Add to the df
	df["new_translations"] = new_translations
	df["substitution_info"] = substituted_all_info
	df["substituted_pairs"] = substituted_only
	df["substituted_words"] = substituted_words
	df["errors"] = error_list

	# Change the working directory once again
	os.chdir("..")


	# Display most common substitutions
	df_substituted = df[df["proper_nouns"] != "0"]
	print(df_substituted.substituted_pairs.value_counts()[:20].to_markdown())
	print("\n\n")

	return df

corrected_df = correct_proper_nouns(test_df)




Alignment started.

Number of aligned sentences: 41


|                                                                |   substituted_pairs |
|:---------------------------------------------------------------|--------------------:|
| 0                                                              |                  33 |
| [('Webr', 'Veber')]                                            |                   2 |
| [('Webra', 'Veber')]                                           |                   1 |
| [('Gasper', 'Gašper'), ('Dusan', 'Dušan'), ('Vukko', 'Vučko')] |                   1 |
| [('Prelešnik', 'Prelesnik'), ('Jernej', 'Jerneja')]            |                   1 |
| [('Marjani', 'Marjana')]                                       |                   1 |
| [("Brglez's", 'Brglez')]                                       |                   1 |
| [('Ludmila', 'Ljudmila')]                                      |                   1 |





In [22]:
corrected_df.head()

,file,sentence_id,text,proper_nouns,length,translation,translation_added_spaces,alignments,new_translations,substitution_info,substituted_pairs,substituted_words,errors
Column1,,,,,,,,,,,,,
443123,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg1.1,"Spoštovani , prosim , da zasedete svoja mesta .",0,9,"Respectable, please take your seats.","Respectable , please take your seats .","{0: 0, 1: 1, 2: 2, 4: 3, 6: 4, 7: 5, 8: 6}","Respectable , please take your seats .",0,0,{},No
443124,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg2.1,V naši sredini pozdravljam predsednika države ...,"{8: ['Boruta', 'Borut'], 9: ['Pahorja', 'Pahor...",19,In our midst I welcome the President of the St...,In our midst I welcome the President of the St...,"{0: 0, 1: 1, 2: 2, 3: 4, 4: 6, 5: 9, 6: 10, 7:...",In our midst I welcome the President of the St...,"[No substitution: ('Borut', 'Borut'), No subst...","[(Webra, Veber)]",{25: 'Webra'},No
443125,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg2.2,Lepo pozdravljena .,0,3,Nicely greeted.,Nicely greeted .,"{0: 0, 1: 1, 2: 2}",Nicely greeted .,[],0,{},No
443126,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg3.1,Besedo dajem spoštovanemu predsedniku Državneg...,"{8: ['Janku', 'Janko'], 9: ['Vebru', 'Veber']}",11,I give the floor to the distinguished Presiden...,I give the floor to the distinguished Presiden...,"{0: 3, 1: 4, 2: 6, 3: 7, 4: 10, 5: 11, 6: 12, ...",I give the floor to the distinguished Presiden...,"[No substitution: ('Janko', 'Janko'), (Webr, V...","[(Webr, Veber)]",{16: 'Webr'},No
443127,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.ana.xml,ParlaMint-SI_2014-08-01-SDZ7-Redna-01.seg3.2,"Izvolite , prosim .",0,4,"Here you go, please.","Here you go , please .","{0: 0, 1: 3, 2: 4, 3: 5}","Here you go , please .",[],0,{},No


In [23]:
# Save the df
corrected_df.to_csv("Parlamint-SI-SlavicMT-sample-corrected.csv", sep="\t")